In [ ]:
#SKU-Level Linear Regression: What Sells Most

#This version aggregates all orders by SKU to get each product’s total quantity sold and average amount.
#Then it fits a linear regression model to see which attributes drive higher total sales.

In [4]:
#SKU-Level Regression: Predicting Product Popularity

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load data
from pathlib import Path
project_root = Path.cwd()
candidates = [
    project_root / "data" / "raw" / "Amazon Sale Report.csv",
    project_root.parent / "data" / "raw" / "Amazon Sale Report.csv"
]
for candidate in candidates:
    if candidate.exists():
        data_path = candidate
        break
else:
    raise FileNotFoundError("Couldn't locate 'data/raw/Amazon Sale Report.csv' from current working directory.")
df = pd.read_csv(data_path)
df.columns = df.columns.str.strip()

# Convert numeric columns
df['Qty'] = pd.to_numeric(df['Qty'], errors='coerce').fillna(0)
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce').fillna(0)

# Aggregate by SKU
agg_df = (
    df.groupby(['SKU', 'Style', 'Category', 'Size', 'Sales Channel', 'Fulfilment', 'fulfilled-by'], dropna=False)
      .agg({'Qty': 'sum', 'Amount': 'mean'})
      .reset_index()
)

# Fill missing text fields
agg_df = agg_df.fillna('Unknown')

# One-hot encode
categorical_cols = ['Category', 'Size', 'Sales Channel', 'Fulfilment', 'fulfilled-by']
agg_encoded = pd.get_dummies(agg_df[categorical_cols], drop_first=True)

# Define X and y
X = pd.concat([agg_encoded, agg_df[['Amount']]], axis=1)
y = agg_df['Qty']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit regression model
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\n✅ SKU Regression Results:")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.4f}")

# Feature importance
coefficients = pd.Series(model.coef_, index=X.columns).sort_values(ascending=False)
print("\n🔹 Top 15 Positive Features (drive higher sales):\n", coefficients.head(15))
print("\n🔹 Top 15 Negative Features (lower sales):\n", coefficients.tail(15))

# Find top-selling SKUs
print("\n🔥 Top 10 Most Popular SKUs:\n")
print(agg_df[['SKU', 'Category', 'Size', 'Qty']].sort_values('Qty', ascending=False).head(10))

C:\Users\anria\AppData\Local\Temp\ipykernel_38236\2571153955.py:22: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_path)



✅ SKU Regression Results:
RMSE: 19.35
R²: 0.0455

🔹 Top 15 Positive Features (drive higher sales):
 Category_Western Dress    14.529545
Category_Set               7.448648
Size_6XL                   7.118155
Category_kurta             6.180175
Category_Top               4.438280
Category_Ethnic Dress      4.022161
Size_M                     3.839542
Size_L                     3.711206
fulfilled-by_Unknown       3.226122
Size_XL                    2.698507
Size_5XL                   2.574258
Size_XXL                   0.953199
Size_4XL                   0.610663
Size_Free                  0.395797
Size_S                     0.163790
dtype: float64

🔹 Top 15 Negative Features (lower sales):
 Size_L                       3.711206
fulfilled-by_Unknown         3.226122
Size_XL                      2.698507
Size_5XL                     2.574258
Size_XXL                     0.953199
Size_4XL                     0.610663
Size_Free                    0.395797
Size_S                       0.163

Overall model

RMSE: 19.35 - SKU-level sales vary a lot; some SKUs sell hundreds while most sell few - normal.

R²: 0.0455 - Product attributes explain only 4.5% of variation in sales.
But the coefficients still show clear patterns of what sells more.

Categories That Strongly Drive Higher Sales

Top positive predictors:



Category:  Western Dress- Strongest positive effect — sells the most;


Sets (Set)- Also strong sellers;


Kurta- Very strong positive driver;


Top- Strong positive;


Ethnic Dress- Moderately positive;

Conclusion:
Our best-performing categories are:
Western Dresses; Sets; Kurtas; Tops; Ethnic Dresses
These should be your priority for restocking.

Categories That Lower Sales


Avoid overstocking:
Sarees, Dupattas, Bottoms

Sizes That Sell Better
Strong positive sizes:

6XL; 5XL; 4XL; XXL; XL; L; M

Summary: What We Should Stock More


Best categories to stock:
Western Dresses;
Sets;
Kurtas;
Tops;
Ethnic Dresses;

Best sizes to stock:
M, L, XL, XXL, 4XL, 5XL, 6XL;

Avoid overstocking:
Sarees;
Dupattas;
Bottoms;
XS;
S;